# Navigation Project

In [1]:
from unityagents import UnityEnvironment
import numpy as np
import torch
from collections import deque
import matplotlib.pyplot as plt
import pickle


Start the environment predefined by the DRLND team for the Navigation project!  
**_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Banana.app"`
- **Windows** (x86): `"path/to/Banana_Windows_x86/Banana.exe"`
- **Windows** (x86_64): `"path/to/Banana_Windows_x86_64/Banana.exe"`
- **Linux** (x86): `"path/to/Banana_Linux/Banana.x86"`
- **Linux** (x86_64): `"path/to/Banana_Linux/Banana.x86_64"`
- **Linux** (x86, headless): `"path/to/Banana_Linux_NoVis/Banana.x86"`
- **Linux** (x86_64, headless): `"path/to/Banana_Linux_NoVis/Banana.x86_64"`


In [2]:
env = UnityEnvironment(file_name="G:\workspace\drlnd\p1_navigation\Banana_Windows_x86_64\Banana.exe")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

The simulation contains a single agent that navigates a large environment.  At each time step, it has four actions at its disposal:
- `0` - walk forward 
- `1` - walk backward
- `2` - turn left
- `3` - turn right

The state space has `37` dimensions and contains the agent's velocity, along with ray-based perception of objects around agent's forward direction.  A reward of `+1` is provided for collecting a yellow banana, and a reward of `-1` is provided for collecting a blue banana. 

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions

action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]

state_size = len(state)
print('States have length:', state_size)

Number of agents: 1
Number of actions: 4
States have length: 37


### 3. Train the agent

In [5]:
from dqn_agent import DqnAgent

agents = [
    DqnAgent(state_size=37, action_size=4, seed=0, use_double=False, use_prio=False, use_dueling=False),
    DqnAgent(state_size=37, action_size=4, seed=0, use_double=False, use_prio=False, use_dueling=True),
    DqnAgent(state_size=37, action_size=4, seed=0, use_double=False, use_prio=True, use_dueling=True),
    DqnAgent(state_size=37, action_size=4, seed=0, use_double=False, use_prio=True, use_dueling=False),
    DqnAgent(state_size=37, action_size=4, seed=0, use_double=True, use_prio=False, use_dueling=False),
    DqnAgent(state_size=37, action_size=4, seed=0, use_double=True, use_prio=False, use_dueling=True),
    DqnAgent(state_size=37, action_size=4, seed=0, use_double=True, use_prio=True, use_dueling=True),
    DqnAgent(state_size=37, action_size=4, seed=0, use_double=True, use_prio=True, use_dueling=False),
]

In [6]:
def run_experiment(agent, idx, n_episodes=1000, max_t=100000, eps_start=1.0, eps_end=0.01, eps_decay=0.987):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    eps = eps_start  
    
    # initialize epsilon
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]
        state = env_info.vector_observations[0]
        score = 0
        for t in range(max_t):
            action = agent.act(state, eps)
            env_info = env.step(action)[brain_name]        # send the action to the environment
            next_state = env_info.vector_observations[0]   # get the next state
            reward = env_info.rewards[0]                   # get the reward
            done = env_info.local_done[0]
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                break 
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        eps = max(eps_end, eps_decay*eps) # decrease epsilon
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
        if np.mean(scores_window)>=13.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
            torch.save(agent.qnetwork_local.state_dict(), 'agent_{}_{}.pth'.format(idx, i_episode-100))
            with open('agent_{}_{}.txt'.format(idx, i_episode-100), 'w') as f:
                f.write(agent.summary())
            break
    return scores

In [7]:
scores = []
for i, agent in enumerate(agents):
    print('\nRunning experiment with agent #{}\n'.format(i))
    scores.append(run_experiment(agent, i))
    pickle.dump(scores, open('scores.dump', 'wb'))
    


Running experiment with agent #0

Episode 100	Average Score: 1.22
Episode 200	Average Score: 7.94
Episode 300	Average Score: 10.98
Episode 400	Average Score: 12.79
Episode 403	Average Score: 13.00
Environment solved in 303 episodes!	Average Score: 13.00

Running experiment with agent #1

Episode 100	Average Score: 1.11
Episode 200	Average Score: 5.16
Episode 300	Average Score: 8.86
Episode 400	Average Score: 12.42
Episode 420	Average Score: 13.09
Environment solved in 320 episodes!	Average Score: 13.09

Running experiment with agent #2

Episode 100	Average Score: 0.72
Episode 200	Average Score: 4.50
Episode 300	Average Score: 9.18
Episode 400	Average Score: 11.76
Episode 463	Average Score: 13.01
Environment solved in 363 episodes!	Average Score: 13.01

Running experiment with agent #3

Episode 100	Average Score: 1.76
Episode 200	Average Score: 7.75
Episode 300	Average Score: 8.76
Episode 400	Average Score: 10.16
Episode 500	Average Score: 12.98
Episode 502	Average Score: 13.05
Environ

In [ ]:
# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

In [ ]:
# load the weights from file
#agent.qnetwork_local.load_state_dict(torch.load('checkpoint.pth'))

env_info = env.reset(train_mode=False)[brain_name]
state = env_info.vector_observations[0]            # get the current state
score = 0 

while True:
    action = agent.act(state)
    env_info = env.step(action)[brain_name]
    next_state = env_info.vector_observations[0]   # get the next state
    reward = env_info.rewards[0]                   # get the reward
    done = env_info.local_done[0]                  # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    if done:                                       # exit loop if episode finished
        break
          
print('Score: {}'.format(score))
env.close()